# summary

1. 전처리 로직에 따라 컬럼 전처리
2. 특이 컬럼
- product_category, product_subcategory, product_modelname : 원핫 적용한 테이블 중 중요도 상위 6개 컬럼 채택, sub과modelname drop
- business_area, business_subarea : 원본 컬럼 drop, 원핫 방식으로 표현된 area 컬럼 중 중요도 상위 3개 컬럼 채택
3. Robust Scaler, Label Encoding, One-Hot Encoding, 구간화로 파생변수 생성 
4. target 클래스 불균형으로 모델링 시 가중치 줌 (1:10으로)
5. CV로 best parameter 구한 후
6. catboostclassifier 모델링
7. Feature Selection

# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 1) 필수 라이브러리

In [36]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import re
from collections import Counter

### 2) 데이터 셋 읽어오기

In [37]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

df = pd.concat([df_train, df_test], axis=0)

In [38]:
df.head() # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted,id
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,1.0,NaN
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,1.0,NaN
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,1.0,NaN
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,1.0,NaN
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,1.0,NaN


## 2. 피처 클렌징 & 엔지니어링

### 1) 컬럼별


#### customer_country

In [39]:
# 1. customer_country
# country 전처리 함수

def pp_customer_country(text):
    country_map = {
        'u.s.a': ['us', 'usa', 'united states', 'ave', 'dr.', 'dr ', 'drive', 'st', 'ct', 'nevada', 'diablo', 'enterprise dr', 'east fl', 'maxine dr', 'mishawum', 'montour', 'kimball', 'highlands', 'hempston', 'bucaramanga', 'florida', 'hampshire', 'ny', 'ohio', 'roadbostonma', 'ca'],
        'italy': ['italy', 'bari'],
        'u.a.e': ['dubai', 'saudi', 'uae'],
        'antigua and barbuda': ['antigua'],
        'turkey': ['türkiye'],
        'brazil': ['são', 'capão', 'aparecida', 'joão pessoa', 'horizonte', 'cuiabá', 'dourados', 'manaus', 'recife', 'janeiro', 'rj', 'paulo'],
        'spain': ['madrid', 'elche', 'agost', 'caceres', 'canarias', 'valencia'],
        'netherlands': ['netherlands', 'curaçao', 'aruba', 'maarten'],
        'vietnam': ['ha noi'],
        'congo': ['congo'],
        'colombia': ['colombia', 'barranquilla', 'carrera', 'cartagena'],
        'india': ['indore', 'lucknow', 'chennai', 'anand', 'bangalore', 'bhilwara', 'gujarat', 'gurgaon', 'hyderabad', 'kerela', 'mumbai', 'odisha', 'pune', 'telangana', 'pradesh'],
        'greece': ['θ'],
        'nigeria': ['benin'],
        'saint kitts and nevis': ['st kitts'],
        'nan': ['', 'nd', 'br', '5555', 'a']
    }

    text = str(text).lower().split('/')[-1].strip()

    for country, keys in country_map.items():
        for k in keys:
            if k in text:
                return country
        
    if '@' in text:
        return 'email'

    if re.search(r'\d{6,}', text) or re.search(r'\d{5,}', text):
        return 'u.s.a'

#     return text

# train, test셋에 전처리 함수 적용
df['customer_country'] = df['customer_country'].apply(pp_customer_country)

#### customer_type, customer_job

In [40]:
def split_word(temp):
    if isinstance(temp, str):  # 문자열인지 확인
        temp = [temp]  # 문자열을 리스트로 변환하여 단일 값도 처리 가능하도록 함
    result = []
    for item in temp:
        if isinstance(item, str):  # 문자열인지 다시 확인
            if "/" in item:
                sen_temp = item.split("/")
            elif "&" in item:
                sen_temp = item.split("&")
            elif "_" in item:
                sen_temp = item.split("_")
            elif "," in item:
                sen_temp = item.split(",")
            elif "." in item:
                sen_temp = item.split(".")
            else:
                sen_temp = [item]  # 위 조건에 해당되지 않으면 그대로 유지
            sen_temp = [word.strip() for word in sen_temp]
            result.append(" ".join(sen_temp))
        else:
            result.append(item)  # 문자열이 아닌 경우 그대로 유지
    return result


In [41]:
def pp_customer_job(row):
    customer_jobs = row['customer_job']
    
    if isinstance(customer_jobs, str):  # 문자열인지 확인
        categories = ['art and design', 'marketing', 'education', 'information technology', 'administrative', 'sales', 'operation', 'program and project management', 'film production', 'finance', 'human resources', 'accounting', 'media and communication', 'legal', 'biomedical', 'construction', 'property', 'R&D','other']
        display_keywords = {
            'art and design': ['arts', 'art', 'arts and design', 'arte y diseño', 'design', 'artist, lead on equipment selection', 'arte_e_design', 'művészet_és_design'],
            'marketing': ['marketing', 'advertising'],
            'education': ['educator', 'education', 'teacher'],
            'information technology': ['engineer', 'develop', 'si', 'it', 'technology', 'chief eng'],
            'administrative': ['administration', 'administración', 'admin', 'administrative'],
            'sales' : ['sales', 'sale', 'vendite', 'értékesítés', 'salesman'],
            'operation': ['operations executive', 'regional director of operations', 'operations manager', 'director of operations', 'strategy & operations specialist', 'facilities and operations', 'operaciones'],
            'program and project management': ['project', 'genel müdür', 'gestión_de_proyectos', 'programm'],
            'film production': ['film production'],
            'finance': ['finance','finanzas', 'finanzen', 'pénzügy', 'vertrieb'],
            'human resources': ['hr','human resources', 'human resource'],
            'accounting': ['account', 'accounting', 'account management', 'account manager'],
            'media and communication' : ['media', 'media e comunicazione', 'média és kommunikáció', 'medios de comunicación', 'medien und kommunikation', 'communication'],
            'legal' : ['legal'],
            'biomedical' : ['medical solution', 'healthcare', 'spécialiste_en_imagerie_médicale', 'medical imaging'], 
            'medical' : ['doctor', 'nurse', 'pathologist', 'profesional de cirugía', 'surgery professional\u200b', 'radiology professional', 'főorvos', 'clinic', 'radiology_professional', 'profesional de radiología', 'cirugano', 'chirurgien', 'surgery professional'],
            'construction' : ['architect', 'project architect', 'contractor' ],
            'property' : ['real estate', 'proprietário(a)'],
            'R&D': ['research', 'R&D', 'r&d', ],
            'other': ['other', 'others', 'otherss']
        }

        for category in categories:
            for keyword in display_keywords[category]:
                if keyword in customer_jobs:
                    return category

    return 'other'


In [42]:
df.loc[:,'customer_job']= split_word(df['customer_job'])
df['customer_job'] = df.apply(pp_customer_job, axis=1)

- 이전 버전

#### historical_existing_cnt

In [43]:
# df['historical_existing_cnt'] = df['historical_existing_cnt'].fillna(0)
# df = df[df['historical_existing_cnt']<1000]

####  inquiry_type

- train set

In [44]:
def pp_inquiry_type_1(df):
    # inquiry_type 열을 소문자로 변환
    df['inquiry_type'] = df['inquiry_type'].str.lower()
    
    # 대체할 문자열과 대상 문자열 정의
    replacement_mapping = {
        '(select id_needs)':'etc',
        'customer suggestions':'customer suggestions',
        'digital platform':'product information',
        'display product':'product information',
        'display textbook and photos':'product information',
        'educational equipments':'product information',
        'estoy buscando para ecuador este producto lg magnit micro led, para un cliente de 138 pulgadas, con envió marítimo.':'product purchase inquiry',
        'etc.':'etc',
        'event inquiry':'etc',
        'evento_sdelestero':'etc',
        'first info and pricing':'pricing and demo request',
        'for school':'product information',
        'high inch 86 / 98 or 110':'product information',
        'hola me pueden cotizar 19 pantallas interactivas de 100 pulgadas entregadas en guayaquil -ecuador.':'quotation and purchase Inquiry',
        'hospital tv':'product information',
        'hotel tv products':'product information',
        'i want to know the details about it':'product information',
        'idb':'product and technical support/consultation',
        'intégrateur historique du george v':'intégrateur historique du george v',
        'led signage':'product information',
        'media inquiry':'customer suggestions',
        'needs':'etc',
        'not specified':'other',
        'oem/odm request':'product and technical support/consultation',
        'one quick:flex':'product information',
        'other_':'other',
        'others':'other',
        'pantallas interactivas para clinicas':'product information',
        'preciso de um monitor médico para radiografia convencional e tomogrtafia.':'product information',
        'probeam precio':'pricing and demo request',
        'purchase or quotation':'purchase',
        'quotation or purchase consultation':'purchase',
        'quotation_':'purchase',
        'quotation_or_purchase_consultation':'purchase',
        'request a demo':'pricing and demo request',
        'request for distributorship':'partnership/distributorship request',
        'request for partnership':'partnership/distributorship request',
        'request for quotation or purchase':'quotation and purchase Inquiry',
        'request for technical consulting':'product and technical support/consultation',
        'sales inquiry':'sales',
        'services':'product and technical support/consultation',
        'solicito apoyo para realizar cotizacion de los dispositivos que ofrecen en la solución one quick: ':'quotation and purchase Inquiry',
        'standalone':'product information',
        'teach':'trainings',
        'technical consultation':'technical',
        'technical support':'technical',
        'technical_consultation':'technical',
        'toi muon tim hieu thong tin ky thuat, gia ca cua sp de su dung':'quotation and purchase Inquiry',
        'tv interactive':'product information',
        'tôi cần tham khảo giá và giải pháp từ lg':'quotation and purchase Inquiry',
        'usage or technical consultation':'product and technical support/consultation',
        'usage_or_technical_consultation':'product and technical support/consultation',
        'video wall':'product and technical support/consultation',
        'vrf':'product information',
        'vui lòng báo giá giúp mình sản phẩm đo thân nhiệt xin cảm ơn':'product information',
        'window facing product':'product information',
        'aio':'product information',
        'product purchase inquiry':'purchase',
        'purchaseor_purchase_consultation':'purchase',
        'quotation and purchase inquiry':'purchase',
        'solicito apoyo para realizar cotizacion de los dispositivos que ofrecen en la solución one quick: ':'purchase',
        'usage or technical':'technical',
        'usage_or_technical':'technical',
        'customer suggestions': 'etc',
        'intégrateur historique du george v': 'etc',
        'solicito apoyo para realizar cotizacion de los dispositivos que ofrecen en la solución\xa0one quick:\xa0': 'purchase',
        'product and technical support/consultation':'product and technical consultation',
        'product and technical/consultation':'product and technical consultation',
        'product and technical consultation':'technical',
        'technical':'product and technical consultation',
        'quotation and purchase inquiry':'purchase',
        'purchase':'quotation and purchase inquiry',
        'product and product and product and product and product and product and technical consultation':'product and technical consultation',
        'quotation and quotation and purchase inquiry inquiry':'quotation and purchase inquiry',
        'quotation and quotation and purchase inquiry inquiry':'quotation and purchase inquiry',
        'product and product and product and product and product and technical consultation':'product and technical consultation',
        'product and product and product and product and technical consultation':'product and technical consultation',
        'quotation and purchase inquiryor_quotation and purchase inquiry_consultation':'quotation and purchase inquiry',
        'quotation or quotation and purchase inquiry consultation':'quotation and purchase inquiry',
        'request for product and product and product and product and product and technical consultation consulting':'product and technical consultation',
        'request for quotation or quotation and purchase inquiry':'quotation and purchase inquiry',
        'usage or product and product and product and product and product and technical consultation':'product and technical consultation',
        'quotation and quotation and purchase inquiry inquiry':'quotation and purchase inquiry',
        'product and product and product and technical consultation':'product and technical consultation',
        'customer suggestions':'etc',
        'quotation and quotation and purchase inquiry inquiry':'product and technical consultation',
    }

    # 대상 문자열에 대해 대체 수행
    for old_str, new_str in replacement_mapping.items():
        # 괄호가 있는 경우
        if '(' in old_str:
            df['inquiry_type'] = df['inquiry_type'].replace(r'\('+old_str.strip('()')+r'\)', new_str, regex=True)
        # 괄호가 없는 경우
        else:
            df['inquiry_type'] = df['inquiry_type'].replace(old_str, new_str, regex=True)

    # inquiry_type 열의 값을 소문자로 변환하여 Counter에 전달
    inquiry_type_counts = Counter(str(value).lower() for value in df['inquiry_type'] if not isinstance(value, float))

    # 총 고유값 개수 확인
    total_unique_values = len(inquiry_type_counts)
    print("총 고유값 개수:", total_unique_values)

    # 각 고유값과 빈도수 출력
    print("고유값 및 빈도수:")
    for value, count in sorted(inquiry_type_counts.items()):
        print(f"'{value}':'{count}'")


    # 최빈값 계산
    mode_value = df['inquiry_type'].mode()[0]

    # 결측치를 최빈값으로 대체
    df['inquiry_type'].fillna(mode_value, inplace=True)
    print('---------------------')
    return df

In [45]:
def pp_inquiry_type_2(df):
    # inquiry_type 열을 소문자로 변환
    df['inquiry_type'] = df['inquiry_type'].str.lower()
    
    # 대체할 문자열과 대상 문자열 정의
    replacement_mapping = {
        'customer suggestions':'etc',
        'quotation and quotation and purchase inquiry inquiry':'quotation and purchase inquiry',
    }

    # 대상 문자열에 대해 대체 수행
    for old_str, new_str in replacement_mapping.items():
        # 괄호가 있는 경우
        if '(' in old_str:
            df['inquiry_type'] = df['inquiry_type'].replace(r'\('+old_str.strip('()')+r'\)', new_str, regex=True)
        # 괄호가 없는 경우
        else:
            df['inquiry_type'] = df['inquiry_type'].replace(old_str, new_str, regex=True)

    # inquiry_type 열의 값을 소문자로 변환하여 Counter에 전달
    inquiry_type_counts = Counter(str(value).lower() for value in df['inquiry_type'] if not isinstance(value, float))

    # 총 고유값 개수 확인
    total_unique_values = len(inquiry_type_counts)
    print("총 고유값 개수:", total_unique_values)
    

    # 각 고유값과 빈도수 출력
    print("고유값 및 빈도수:")
    for value, count in sorted(inquiry_type_counts.items()):
        print(f"'{value}':'{count}'")

    # 최빈값 계산
    mode_value = df['inquiry_type'].mode()[0]

    # 결측치를 최빈값으로 대체
    df['inquiry_type'].fillna(mode_value, inplace=True)
    print('---------------------')
    return df


In [46]:
df = pp_inquiry_type_1(df)
df = pp_inquiry_type_2(df)

총 고유값 개수: 11
고유값 및 빈도수:
'customer suggestions':'4'
'etc':'246'
'other':'1303'
'partnership/distributorship request':'436'
'pricing and demo request':'196'
'product and technical consultation':'2434'
'product information':'1325'
'quotation and purchase inquiry':'45621'
'quotation and quotation and purchase inquiry inquiry':'235'
'sales':'10081'
'trainings':'456'
---------------------
총 고유값 개수: 9
고유값 및 빈도수:
'etc':'250'
'other':'1303'
'partnership/distributorship request':'436'
'pricing and demo request':'196'
'product and technical consultation':'2434'
'product information':'1325'
'quotation and purchase inquiry':'48089'
'sales':'10081'
'trainings':'456'
---------------------


#### product_category

In [47]:
def pp_product_category(row):
    category_dict = {
        'display': ['Display','video', 'signage', 'Signage', '43uh5f', '49uh', '49vl', '49vl', '55vm', '86uh', '98uh', 'display', 'uh5f', 'fhd', 'gsca',
                    'gscd', 'brightness', 'laec', 'leadallin', 'bloc', 'one:quick', 'lsca', 'screen', '顯示屏',
                    'one quick', 'onequick', 'led', 'lcd', 'oled', 'ultra stretch series', 'ur640', 'videowall', 'videwall', 'Monitors',
                    'virtual production', 'window facing', 'pol', '28mq780', 'monitor', 'UM3DG', 'UN880', 'MQ780', 'QP88D', 'ERGO', 'UH', '65EP5G',
                    'UL3J', 'WN780', 'svh7f', 'uh', '32SM5J', 'inch', 'essential series'],
        
        'tv': ['43uq751c0sb', 'lq621cbsb', '43us660h', '50uq801', '50us660', 'tv', 'TV', 'us670', 'pro centric', 'procentric', 'smart tv', '電視', 'CT5WJ', 'HT3WJ', 'uq801c0sb', 'uq751c0sf',
               'us660h0sd'],
        'temperature': ['definir', 'thermodynamic water heater', 'ac rumah', 'acondicionado', 'condicionado' 'vrf', 'aquecimento',
                        'calefacción', 'chiller', 'climatiseur', 'heating', 'isıtma', 'split', 'magnit', 'inverter', 'multi', 'aio',
                        'ogrzewanie', 'pendingin', 'rac', 'cac', 'conditioner', 'sac', 'scroll compressor', 'soğutucu',
                        'system ac', 'réfrigérant', 'cassete inverter', 'điều hòa', 'אחר', 'חימום', 'מזגנים ', 'מרובה ', 'تكييف وتبريد', 'تكييفات',
                        'เครื่องปรับอากาศเผื่อที่อยู่อาศัย', 'مبرد', 'reversible ac', 'single package', 'حلول التدفئة', 'פיצול מרובה', 'unitario'],
        'air': ['ahu', 'cac', 'air solution', 'air solution', 'ventilation', 'aircare', 'vrf'],
        'beam': ['bu50nst', 'projector', 'Beam', 'Projector'],
        'beauty' : ['beauty'],
        'board': ['createboard', 'idb', 'borad', 'tr3', '55tc3d'],
        'cloud': ['cloud', 'Cloud', 'id', 'cloud device', 'Thin Clients', 'Zero Clients'],
        'pc': ['laptop', 'pc', 'Laptops'],
        'software': ['pro centric', 'procentric', 'software', 'software solution', 'webos', '軟體', 'pro:centric', 'SuperSign'],
        'refrigerator': ['refrigerator'],
        'robot': ['robot'],
        'energy': ['solar', 'energy', 'energy storage system'],
        'other': ['autre', 'inne', 'khác', 'etc.', 'lainnya', 'not specified', 'other', 'Other', 'otros', 'outros', 'ฯลฯ', '其他'],
        'parts_and_accessories' : ['parts', 'accessories', 'Accessories', 'Antennas'],
        'plug' : ['SC-00DA'],
        'appliances' : ['washing machine', 'dryer', 'vb.', 'vacuum'],
        'hospital' : ['Medical', 'medical', 'hospital', 'X-ray', 'HN713D', 'HQ513D', 'surgical', 'จอภาพเพื่อการวินิจฉัย', 'จอภาพสำหรับการตรวจสอบทางคลินิก'],
        'services' : ['support', 'care program', 'Enqiry', 'inquiry', 'service', 'error']
    }
    
    for category, keywords in category_dict.items():
        for keyword in keywords:
            if keyword in row['product_category']:
                row[category] = 1
                break

    return row


In [48]:
products = df[['product_category', 'product_subcategory', 'product_modelname']]
products = products.assign(com=products.apply(lambda row: ''.join(str(cell).replace('nan', '') for cell in row), axis=1))

category_table = pd.DataFrame(products['com'])
category_table.rename(columns={'com': 'product_category'}, inplace=True)
category_table[['display', 'tv', 'temperature', 'air', 'beam', 'beauty', 'board', 'cloud', 'pc', 'software', 'refrigerator', 'robot', 'energy', 'other', 'parts_and_accessories', 'plug', 'appliances', 'hospital', 'services']] = 0
category_table = category_table.fillna('None')
category_table = category_table.apply(pp_product_category, axis=1)

카테고리 테이블 변수 + is_converted feature_importances

Feature Importances:
- display: 29.18399582507811
- tv: 19.364034410354535
- hospital: 10.837545324510517
- temperature: 9.26037935140265
- cloud: 6.653500225631451
- services: 5.3337207309981425
- air: 4.641667971098805
- energy: 3.5587050590332763
- other: 2.7262252053739178
- beam: 1.8745732673249074
- pc: 1.3936238090659425
- parts_and_accessories: 1.2108271669736377
- software: 0.9954090684498512
- appliances: 0.7399095693246863
- board: 0.6383874878258683
- plug: 0.6360597270298045
- refrigerator: 0.5915976131280704
- robot: 0.311556815769963
- beauty: 0.04828137162589573

In [49]:
df = pd.concat([df, category_table.iloc[:, 1:]], axis=1)


####  customer_position

In [50]:
# 17) customer_position

def pp_cp(text):
    text = text.replace('/', ' ').lower()

    position_map = {
        'executive': ['ceo', 'executive', 'president', 'entrepre', 'found', 'gerente', 'proprie', 'genel', 'chairman', 'boss', 'vp'],
        'trainee': ['intern', 'unpaid'],
        'vice president': ['vicepresident'],
        'engineering': ['installer', 'técnico', 'tech', 'desarrollo'],
        'lecturer': ['profess', 'lecture', 'educa', 'teacher', 'principal', 'academic', 'prof', 'faculty', 'hon dean', 'exam', 'pgt'],
        'none': ['bulgaria', 'not applicable'],
        'manufacturer': ['manufacturer'],
        'customer': ['user'],
        'entry level': ['entry'],
        'consultant': ['consult', 'career coach'],
        'decision maker': ['decision'],
        'director': ['director'],
        'hospital': ['főorvos', 'tierarzt', 'medical'],
        'exhibition': ['exhibition'],
        'employee': ['this is', 'mindenes', 'no influence'],
        'sales': ['sales'],
        'research': ['research', 'pathologist'],
        'others': ['other'],
        'partner': ['distributor', 'partner'],
        'management': ['manage']
    }

    for position, keywords in position_map.items():
        for keyword in keywords:
            if keyword in text:
                return position

    return text

df['customer_position'] = df['customer_position'].apply(pp_cp)

#### expected_timeline

In [51]:
# expected_timeline

def pp_et(text):
    text = str(text).replace('_', ' ').replace('.','').replace(' - ', ' ~ ')
    if text not in ['nan', 'less than 3 months', '3 months ~ 6 months', 'more than a year','9 months ~ 1 year', '6 months ~ 9 months', 'less than 6 months', 'etc','being followed up']:
        return 'memo'
    return text

df['expected_timeline'] = pd.DataFrame(df['expected_timeline'].apply(pp_et))

## 3. 데이터 전처리

### 1) 스케일링

In [52]:
numeric_features = ['com_reg_ver_win_rate', 'historical_existing_cnt', 'lead_desc_length', 'ver_win_rate_x', 'ver_win_ratio_per_bu']

from sklearn.preprocessing import RobustScaler

# RobustScaler 생성합니다.
scaler = RobustScaler()

# 선택한 변수들에 대해 RobustScaler 적용합니다.
df[numeric_features] = scaler.fit_transform(df[numeric_features])


### 2) 레이블 인코딩

In [20]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [21]:
# 레이블 인코딩할 칼럼들

label_columns = [
    "bant_submit",
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
    'lead_desc_length'
]

for col in label_columns:
    df[col] = label_encoding(df[col])
    


### 3) 원핫인코딩 

In [22]:
import pandas as pd

# 분석할 열 리스트
columns_to_encode = ['business_area', 'response_corporate', 'expected_timeline', 'enterprise']

# 결과를 저장할 리스트
encoded_dfs = []

# 각 열에 대해 원핫인코딩을 수행하고 결과를 리스트에 저장합니다.
for column in columns_to_encode:
    # pd.get_dummies()를 사용하여 해당 열을 원핫인코딩합니다.
    one_hot_encoded = pd.get_dummies(df[column], dtype=int, prefix=column)
    # 원핫인코딩된 결과를 원본 데이터프레임에 추가합니다.
    df = pd.concat([df, one_hot_encoded], axis=1)

# 결과를 확인합니다.
df

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,expected_timeline_2,expected_timeline_3,expected_timeline_4,expected_timeline_5,expected_timeline_6,expected_timeline_7,expected_timeline_8,expected_timeline_9,enterprise_0,enterprise_1
0,4,7,0,0.346149,32160,10,0,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,1,0
1,4,7,0,0.346149,23122,10,0,0.444444,NaN,NaN,...,0,0,0,1,0,0,0,0,1,0
2,4,7,0,0.786046,1755,10,0,7.777778,NaN,NaN,...,0,0,0,1,0,0,0,0,1,0
3,4,7,0,0.786046,4919,10,0,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,1,0
4,4,7,0,0.786046,17126,29,0,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,2,7,0,NaN,40292,33,0,0.333333,NaN,NaN,...,0,0,0,0,0,0,0,1,1,0
5267,1,13,3,NaN,47466,33,0,-0.222222,NaN,NaN,...,0,0,0,0,0,0,0,1,1,0
5268,3,7,0,NaN,46227,29,0,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,1,0
5269,0,13,3,NaN,45667,9,1,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,1


### 4) 구간화

In [23]:
import pandas as pd

# 분석할 열 리스트
columns_to_encode = ['com_reg_ver_win_rate', 'lead_desc_length', 'ver_win_ratio_per_bu']

# 각 열에 대해 원핫인코딩을 수행하고 결과를 리스트에 저장합니다.
encoded_dfs = []
for column in columns_to_encode:
    # pd.qcut()으로 현재 열을 구간별로 나눕니다.
    q = 5
    bins = pd.qcut(df[column], q=q)
    # 각 구간을 원핫인코딩합니다.
    one_hot_encoded = pd.get_dummies(bins, dtype=int)
    # 각 컬럼의 이름을 변경합니다.
    new_column_names = [f'{column}_bin_{i}' for i in range(1, q+1)]  # 수정된 부분
    one_hot_encoded.columns = new_column_names
    # 결과를 리스트에 저장합니다.
    encoded_dfs.append(one_hot_encoded)

# 원핫인코딩된 결과를 합칩니다.
one_hot_table = pd.concat(encoded_dfs, axis=1)
df = pd.concat([df, one_hot_table], axis=1)

### 5) 필요없는 컬럼 drop 

In [24]:
df = df.drop(['customer_country.1', 'product_category', 'product_subcategory', 'product_modelname', 'business_area', 'business_subarea'], axis = 1)


In [23]:
df = df.drop(['robot', 'board', 'appliances', 'services', 'response_corporate_6', 'response_corporate_49', 'response_corporate_4', 'response_corporate_39', 'response_corporate_38', 'response_corporate_29', 'response_corporate_27', 'response_corporate_23', 'response_corporate_13', 'response_corporate_1', 'expected_timeline_6', 'expected_timeline_4', 'energy', 'beauty'], axis = 1)


In [24]:
df = df.drop(['response_corporate_52', 'response_corporate_44', 'response_corporate_25', 'response_corporate_20', 'refrigerator', 'plug', 'expected_timeline_9', 'expected_timeline_5', 'enterprise_0', 'business_area_7', 'business_area_5', 'business_area_10', 'business_area_1'], axis=1)

In [25]:
df = df.drop(['com_reg_ver_win_rate_bin_4', 'com_reg_ver_win_rate_bin_5', 'beam', 'response_corporate_14', 'business_area_12', 'response_corporate_8', 'response_corporate_5', 'response_corporate_30', 'response_corporate_37', 'business_area_3', 'expected_timeline_2', 'response_corporate_0', 'response_corporate_11', 'business_area_2', 'response_corporate_7', 'business_area_9', 'business_area_0'], axis=1)

In [26]:
df = df.drop(['cloud', 'business_area_8', 'response_corporate_45', 'business_area_11', 'response_corporate_19', 'expected_timeline_0', 'lead_desc_length_bin_1', 'it_strategic_ver', 'idit_strategic_ver'], axis=1)

In [27]:
df = df.drop([ 'pc', 'response_corporate_2'], axis=1)

In [25]:
df.shape

(64570, 136)

### 6) train, test 셋 할당

In [26]:
df_train = df[df['id'].isnull()].drop(['id'], axis=1)
df_test = df[~df['id'].isnull()]

In [27]:
df_train.shape, df_test.shape

((59299, 135), (5271, 136))

In [28]:
train_data = df_train.drop(columns=['is_converted']).copy()
test_data = df_test.copy()

target = df_train['is_converted'].copy()

### 7) 학습, 검증 데이터 분리

In [34]:
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import classification_report

In [35]:
X_train, X_validation, y_train, y_validation = train_test_split(train_data, target, train_size=0.8, random_state=42, shuffle=True) # train test split
train_pool = Pool(X_train,y_train)
eval_pool = Pool(X_validation , y_validation)
test_pool = Pool(data=test_data)

## 4-1. 교차검증 Cross validation -> 하이파파라미터튜닝

In [34]:
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import KFold


# random sampler
sampler = TPESampler(seed=10, consider_prior=True)

# 함수 정의

def objective(trial):
    
    param = {
      'random_state' : 42,
      'random_strength' : trial.suggest_float("random_strength", 0.3, 1),
      'colsample_bylevel' : trial.suggest_float('colsample_bylevel', 0.5, 1),
#       'iterations' : trial.suggest_int("iterations", 1000, 2000),
      'l2_leaf_reg' : trial.suggest_float('l2_leaf_reg', 0.5, 2),
      'learning_rate' : trial.suggest_float('learning_rate', 0.015, 0.025),
      'max_depth' : trial.suggest_int("max_depth", 4, 7),
#       "num_trees": trial.suggest_int("num_trees", 100, 1000),
      'subsample': trial.suggest_float("subsample", 0.5, 1.0),
      'class_weights' : {'False': 1,
                         'True': trial.suggest_int('class_weight_true', 9, 15)}
    } 
    
    # 모델 생성
    model = CatBoostClassifier(**param)
    #10 k fold
    f1_list = []
    kf = KFold(n_splits=10)
    #Train_data를 다시 학습 세트와 검증 세트로 분리
    for tr_index,val_index in kf.split(train_data):
        X_train, y_train = train_data.iloc[tr_index], target.iloc[tr_index]
        X_valid , y_valid = train_data.iloc[val_index], target.iloc[val_index]
        #학습 수행
        model = model.fit(X_train,y_train, eval_set=[(X_train,y_train),(X_valid,y_valid)],
                           verbose=False, early_stopping_rounds=1000)                         
        # 평가지표 바꾸는 것도 가능
        y_pred = model.predict(X_valid)
        y_pred = (y_pred == 'True')
        
        f1_list.append(f1_score(y_valid, y_pred, average='binary'))
    return np.mean(f1_list)

optuna_cbrm = optuna.create_study(direction='maximize', sampler=sampler)
optuna_cbrm.optimize(objective, n_trials = 100)

[I 2024-02-25 17:15:37,533] A new study created in memory with name: no-name-b43a6382-0dbf-40f1-a2dc-f3ab2d80ffb7
[I 2024-02-25 17:16:41,469] Trial 0 finished with value: 0.586171863011349 and parameters: {'random_strength': 0.8399244502867222, 'colsample_bylevel': 0.5103759746797008, 'l2_leaf_reg': 1.4504723523894132, 'learning_rate': 0.02248803882538612, 'max_depth': 5, 'subsample': 0.6123983227654238, 'class_weight_true': 10}. Best is trial 0 with value: 0.586171863011349.
[I 2024-02-25 17:18:01,168] Trial 1 finished with value: 0.5912274007343422 and parameters: {'random_strength': 0.8323714985392712, 'colsample_bylevel': 0.5845554182812678, 'l2_leaf_reg': 0.6325097212610153, 'learning_rate': 0.021853598183677974, 'max_depth': 7, 'subsample': 0.5019741331639572, 'class_weight_true': 12}. Best is trial 1 with value: 0.5912274007343422.
[I 2024-02-25 17:19:29,231] Trial 2 finished with value: 0.5788999321964919 and parameters: {'random_strength': 0.8688346731564793, 'colsample_byleve

In [35]:
cbrm_trial = optuna_cbrm.best_trial
cbrm_trial_params = cbrm_trial.params
print(f'Best Trial: score {cbrm_trial.value},\nparams {cbrm_trial_params}')


Best Trial: score 0.6124951539198713,
params {'random_strength': 0.3253288078690793, 'colsample_bylevel': 0.9451193744577909, 'l2_leaf_reg': 1.8203398287370274, 'learning_rate': 0.02201255064174177, 'max_depth': 7, 'subsample': 0.897182325749712, 'class_weight_true': 9}


## 4-2. 모델링 

In [62]:
#Optuna에서 가져온 최적의 파라미터들로 모델 학습
params = {
          'class_weights' : {'False': 1, 'True': 12},
          'learning_rate': 0.02,
          'depth' : 5,
          'iterations': 2000,
          'loss_function' : 'Logloss',
          'custom_metric' : ['F1', 'Logloss'],
#           'eval_metric':'F1',
          'early_stopping_rounds':1000,
          'use_best_model': True,
          'random_seed': 22,
          'verbose':200,
          'random_strength' : 0.3,
          'colsample_bylevel' : 0.9451193744577909,
          'l2_leaf_reg' : 1.9,
          'subsample' : 0.897182325749712
    
          }

model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=eval_pool, use_best_model=True)

0:	learn: 0.6712567	test: 0.6717152	best: 0.6717152 (0)	total: 12.1ms	remaining: 24.2s
200:	learn: 0.2296117	test: 0.2433328	best: 0.2433328 (200)	total: 1.52s	remaining: 13.6s
400:	learn: 0.1888209	test: 0.2084424	best: 0.2084424 (400)	total: 2.9s	remaining: 11.6s
600:	learn: 0.1682350	test: 0.1948846	best: 0.1948846 (600)	total: 4.27s	remaining: 9.94s
800:	learn: 0.1519502	test: 0.1854458	best: 0.1854458 (800)	total: 5.64s	remaining: 8.44s
1000:	learn: 0.1401763	test: 0.1798309	best: 0.1798110 (998)	total: 7.01s	remaining: 7s
1200:	learn: 0.1305151	test: 0.1760507	best: 0.1760507 (1200)	total: 8.38s	remaining: 5.58s
1400:	learn: 0.1222166	test: 0.1732058	best: 0.1732058 (1400)	total: 9.74s	remaining: 4.17s
1600:	learn: 0.1150346	test: 0.1709212	best: 0.1709212 (1600)	total: 11.1s	remaining: 2.77s
1800:	learn: 0.1088501	test: 0.1689729	best: 0.1689729 (1800)	total: 12.5s	remaining: 1.38s
1999:	learn: 0.1031385	test: 0.1682950	best: 0.1682950 (1999)	total: 13.9s	remaining: 0us

bestTes

In [63]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [64]:
y_pred = model.predict(eval_pool)# 검증 데이터에 대한 예측
y_pred = (y_pred == 'True')

get_clf_eval(y_validation, y_pred)  # 모델의 성능 평가

오차행렬:
 [[  907    78]
 [  553 10322]]

정확도: 0.9468
정밀도: 0.6212
재현율: 0.9208
F1: 0.7419


## 4-3. feature importance

In [65]:
# feature importances 계산
feature_importances = model.get_feature_importance()

# 각 특성의 중요도와 변수명을 튜플로 저장
importance_tuples = [(importance, feature_name) for importance, feature_name in zip(feature_importances, X_train.columns)]

# 중요도를 기준으로 정렬
importance_tuples.sort(reverse=True)

# 중요도와 변수명 출력
print("Feature Importances:")
lst = []
for importance, feature_name in importance_tuples:
    lst.append(feature_name)
    print(f"{feature_name}: {importance}")

Feature Importances:
customer_idx: 62.53777357936145
lead_owner: 11.205898378748698
customer_type: 4.433901905825603
inquiry_type: 2.312281442666026
response_corporate: 2.1436321509722593
business_unit: 1.9914311935478377
lead_desc_length: 1.726965295498842
response_corporate_50: 1.2257881709853529
com_reg_ver_win_rate: 1.0088991480903937
response_corporate_33: 0.819920718213638
response_corporate_21: 0.714050173859962
historical_existing_cnt: 0.6956475876359433
customer_country: 0.565092253732148
ver_win_rate_x: 0.5643870765248419
air: 0.5265837566666007
response_corporate_43: 0.506265788043915
customer_job: 0.49103045830004427
bant_submit: 0.4351506792959448
expected_timeline: 0.3795315205813491
response_corporate_32: 0.3606885788064219
customer_position: 0.36040055788456804
lead_desc_length_bin_2: 0.30699175720280447
ver_win_ratio_per_bu: 0.2823649713523872
response_corporate_36: 0.2445393198266972
temperature: 0.2357912704824552
expected_timeline_7: 0.19127173866939856
response_cor

## 5. 제출하기

### 1) 테스트 데이터 예측

In [66]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [67]:
test_pred = model.predict(x_test.fillna(0))
sum(test_pred == 'True') # True로 예측된 개수

1879

### 2) 제출 파일 작성

In [68]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)